In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import molsysmt as msm
from simtk import unit

# Adding solvent

In [ ]:
pdbfile = msm.demo_systems.files['metenkephalin.pdb']

In [ ]:
molecular_system = msm.convert(pdbfile)

In [ ]:
msm.info(molecular_system)

In [ ]:
n_waters, n_ions = msm.get(molecular_system, target='system',
                           n_waters=True, n_ions=True)
print('The molecular system has {} waters and {} ions.'.format(n_waters, n_ions))

In [ ]:
molecular_system = msm.remove_hydrogens(molecular_system)

In [ ]:
molecular_system_solvated = msm.solvate(molecular_system, box_geometry='cubic',
                                        clearance=14.0*unit.angstroms, water='TIP3P',
                                        verbose=False)

In [ ]:
msm.info(molecular_system_solvated)

In [ ]:
n_waters, n_ions = msm.get(molecular_system_solvated, target='system',
                           n_waters=True, n_ions=True)
print('The molecular system has {} waters and {} ions.'.format(n_waters, n_ions))

With Openmm:

In [ ]:
molecular_system = msm.convert(pdbfile)

In [ ]:
msm.get(molecular_system, selection='atom_type=="H"', n_atoms=True)

In [ ]:
#molecular_system_solvated = msm.solvate(molecular_system, box_geometry='cubic',
#                                        clearance=14.0*unit.angstroms, water='TIP3P',
#                                        engine='OpenMM', verbose=False)

## Octahedral

In [ ]:
pdbfile = msm.demo_systems.files['1tcd.pdb']

In [ ]:
molecular_system = msm.convert(pdbfile)

In [ ]:
msm.info(molecular_system)

In [ ]:
n_waters, n_ions = msm.get(molecular_system, target='system',
                           n_waters=True, n_ions=True)
print('The molecular system has {} waters and {} ions.'.format(n_waters, n_ions))

In [ ]:
n_Hs = msm.get(molecular_system, selection='atom_type=="H"', n_atoms=True)
print(n_Hs)

In [ ]:
#molecular_system_solvated = msm.solvate(molecular_system, box_geometry='cubic',
#                                        clearance=14.0*unit.angstroms, water='TIP3P',
#                                        verbose=False)

In [ ]:
msm.info(molecular_system_solvated)